In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
from pathlib import Path

# Configuration matplotlib
plt.style.use('default')
plt.rcParams['figure.figsize'] = (12, 8)

# Chemins
project_root = Path('.').parent
data_dir = project_root / "data"

# Classes
CLASSES = ['cat', 'dog', 'house', 'car', 'tree']
CLASS_EMOJIS = ['🐱', '🐶', '🏠', '🚗', '🌳']

print("📦 Imports terminés!")
print(f"📁 Dossier data: {data_dir}")
print(f"🎯 Classes: {CLASSES}")

# Vérification des fichiers
for class_name in CLASSES:
    filepath = data_dir / f"{class_name}.npy"
    if filepath.exists():
        print(f"✅ {class_name}.npy trouvé")
    else:
        print(f"❌ {class_name}.npy manquant")


In [ ]:
# Chargement et analyse des données
data_stats = {}

print("📊 ANALYSE DES DONNÉES")
print("=" * 40)

for i, class_name in enumerate(CLASSES):
    filepath = data_dir / f"{class_name}.npy"
    
    # Charger les données
    data = np.load(filepath)
    
    # Statistiques
    data_stats[class_name] = {
        'emoji': CLASS_EMOJIS[i],
        'count': len(data),
        'shape': data.shape,
        'size_mb': filepath.stat().st_size / 1024 / 1024,
        'min': data.min(),
        'max': data.max(),
        'mean': data.mean()
    }
    
    stats = data_stats[class_name]
    print(f"{stats['emoji']} {class_name.upper()}")
    print(f"   📊 Images: {stats['count']:,}")
    print(f"   📏 Shape: {stats['shape']}")
    print(f"   💾 Taille: {stats['size_mb']:.1f} MB")
    print(f"   🎨 Pixels: min={stats['min']}, max={stats['max']}, mean={stats['mean']:.1f}")
    print()

# Résumé global
total_images = sum(stats['count'] for stats in data_stats.values())
total_size = sum(stats['size_mb'] for stats in data_stats.values())

print("🌟 RÉSUMÉ GLOBAL")
print("-" * 20)
print(f"📊 Total images: {total_images:,}")
print(f"💾 Total taille: {total_size:.1f} MB")
print(f"📏 Format images: 28x28 (784 pixels)")
print(f"🎨 Valeurs pixels: 0-255 (noir=255, blanc=0)")


In [ ]:
# Visualisation d'exemples de dessins
def show_examples(num_examples=5):
    """Affiche des exemples de dessins pour chaque classe"""
    
    fig, axes = plt.subplots(len(CLASSES), num_examples, figsize=(15, 12))
    fig.suptitle('🎨 Exemples de dessins Quick Draw', fontsize=16, fontweight='bold')
    
    for class_idx, class_name in enumerate(CLASSES):
        # Charger les données de cette classe
        filepath = data_dir / f"{class_name}.npy"
        data = np.load(filepath)
        
        # Prendre des exemples aléatoires
        random_indices = np.random.choice(len(data), num_examples, replace=False)
        
        for example_idx in range(num_examples):
            ax = axes[class_idx, example_idx]
            
            # Obtenir l'image et la redimensionner en 28x28
            img = data[random_indices[example_idx]].reshape(28, 28)
            
            # Afficher (inverser les couleurs pour un fond blanc)
            ax.imshow(255 - img, cmap='gray')
            ax.axis('off')
            
            # Titre pour la première colonne seulement
            if example_idx == 0:
                emoji = CLASS_EMOJIS[class_idx]
                ax.set_title(f'{emoji} {class_name.title()}', 
                           fontsize=12, fontweight='bold', pad=10)
    
    plt.tight_layout()
    plt.show()

print("🖼️ VISUALISATION DES EXEMPLES")
print("=" * 30)
show_examples(6)


In [ ]:
# Distribution des classes et préparation
print("📊 DISTRIBUTION DES CLASSES")
print("=" * 30)

# Graphique de distribution
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Graphique en barres
class_names = list(data_stats.keys())
class_counts = [data_stats[name]['count'] for name in class_names]
class_labels = [f"{CLASS_EMOJIS[i]} {name.title()}" for i, name in enumerate(class_names)]

bars = ax1.bar(class_labels, class_counts, color=['#FF6B6B', '#4ECDC4', '#45B7D1', '#96CEB4', '#FFEAA7'])
ax1.set_title('Nombre d\'images par classe', fontsize=14, fontweight='bold')
ax1.set_ylabel('Nombre d\'images')
ax1.tick_params(axis='x', rotation=45)

# Ajouter les valeurs sur les barres
for bar, count in zip(bars, class_counts):
    height = bar.get_height()
    ax1.text(bar.get_x() + bar.get_width()/2., height + 1000,
             f'{count:,}', ha='center', va='bottom', fontweight='bold')

# Graphique en camembert
ax2.pie(class_counts, labels=class_labels, autopct='%1.1f%%', startangle=90,
        colors=['#FF6B6B', '#4ECDC4', '#45B7D1', '#96CEB4', '#FFEAA7'])
ax2.set_title('Répartition des classes', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.show()

# Recommandations pour l'équilibrage
print("💡 RECOMMANDATIONS POUR L'ENTRAÎNEMENT")
print("-" * 40)

min_count = min(class_counts)
max_count = max(class_counts)
imbalance_ratio = max_count / min_count

print(f"📊 Classe la plus représentée: {max_count:,} images")
print(f"📉 Classe la moins représentée: {min_count:,} images") 
print(f"⚖️ Ratio de déséquilibre: {imbalance_ratio:.1f}x")

if imbalance_ratio > 1.5:
    print("⚠️ Déséquilibre détecté ! Options:")
    print("   1. Utiliser le même nombre d'images pour toutes les classes")
    print("   2. Appliquer des poids aux classes lors de l'entraînement")
    print("   3. Augmentation de données pour les classes sous-représentées")
    
    # Suggestion d'équilibrage
    balanced_count = min_count
    print(f"\n✅ Suggestion: Utiliser {balanced_count:,} images par classe")
    print("   -> Dataset équilibré de", f"{balanced_count * len(CLASSES):,} images total")
else:
    print("✅ Les classes sont bien équilibrées !")

print("\n🚀 PROCHAINES ÉTAPES")
print("-" * 20)
print("1. Preprocessing: normalisation et redimensionnement")
print("2. Split train/validation/test") 
print("3. Architecture CNN adaptée à 28x28")
print("4. Entraînement avec monitoring TensorBoard")
print("5. Évaluation et optimisation")

print("\n✨ Données prêtes pour l'entraînement ! ✨")
